<a href="https://colab.research.google.com/github/kundanvallala111-coder/kundan-vallala/blob/main/Mythology_NLP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!mkdir -p project/src project/data


In [2]:
!pip install sentence-transformers faiss-cpu streamlit pyngrok==5.2.1


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 761.3/761.3 kB 19.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.6/23.6 MB 58.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.2/10.2 MB 74.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 67.4 MB/s eta 0:00:00
  Created wheel for pyngrok: filename=pyngrok-5.2.1-py3-none-any.whl size=19772 sha256=4f03e352eefa96cf1184a94e3fce7771c729f39c407c124b216f818707c81bb5
  Stored in directory: /root/.cache/pip/wheels/e5/44/6c/7c771c9ed861d73dbf63f7d934e16c079e76dc207402d81f2e
Successfully built pyngrok


In [3]:
texts = [
    "Lord Rama is the protagonist of the Ramayana. He is the 7th avatar of Vishnu.",
    "Sita is the wife of Rama and an avatar of Goddess Lakshmi.",
    "Lakshmana is Rama's brother and symbol of loyalty.",
    "Krishna is the 8th avatar of Vishnu and speaker of the Bhagavad Gita.",
    "Arjuna is a Pandava prince and student of Dronacharya.",
    "The Mahabharata describes the Kurukshetra war between Pandavas and Kauravas.",
    "Hanuman is a devotee of Rama known for strength and devotion."
]

import json
with open("project/data/mythology.json", "w") as f:
    json.dump({"texts": texts}, f, indent=4)


In [4]:
from sentence_transformers import SentenceTransformer
import numpy as np
import json

model = SentenceTransformer("all-MiniLM-L6-v2")

with open("project/data/mythology.json") as f:
    db = json.load(f)

embeddings = model.encode(db["texts"]).astype("float32")
np.save("project/data/embeddings.npy", embeddings)


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [5]:
import faiss
import numpy as np

emb = np.load("project/data/embeddings.npy")

index = faiss.IndexFlatL2(emb.shape[1])
index.add(emb)

faiss.write_index(index, "project/data/index.faiss")


In [6]:
%%writefile project/src/backend.py
import faiss
import json
import numpy as np
from sentence_transformers import SentenceTransformer

# Load model + data
model = SentenceTransformer("all-MiniLM-L6-v2")
index = faiss.read_index("project/data/index.faiss")

with open("project/data/mythology.json") as f:
    db = json.load(f)

# Query function
def answer_query(query, top_k=3):
    q_emb = model.encode([query]).astype("float32")
    D, I = index.search(q_emb, top_k)

    retrieved = [db["texts"][idx] for idx in I[0]]

    # Basic filtering
    key = query.lower().split()[0]
    filtered = [t for t in retrieved if key in t.lower()]

    final = filtered if filtered else retrieved

    # Limit output
    return " ".join(final)[:350]


Writing project/src/backend.py


In [7]:
%%writefile project/src/app.py
import streamlit as st
from backend import answer_query

st.title("Mythology Chatbot")

query = st.text_input("Ask something about Hindu Mythology:")

if query:
    answer = answer_query(query)
    st.write(answer)


Writing project/src/app.py


In [8]:
!streamlit run project/src/app.py &>/content/logs.txt &


In [9]:
!ngrok config add-authtoken 35xztwBiPxqnzZsqsr7LPquj6On_3WKiQVHVSzhckNzK1m737


NAME:
   ngrok - tunnel local ports to public URLs and inspect traffic

DESCRIPTION:
    ngrok exposes local networked services behinds NATs and firewalls to the
    public internet over a secure tunnel. Share local websites, build/test
    webhook consumers and self-host personal services.
    Detailed help for each command is available with 'ngrok help <command>'.
    Open http://localhost:4040 for ngrok's web interface to inspect traffic.

EXAMPLES:
    ngrok http 80                    # secure public URL for port 80 web server
    ngrok http -subdomain=baz 8080   # port 8080 available at baz.ngrok.io
    ngrok http foo.dev:80            # tunnel to host:port instead of localhost
    ngrok http https://localhost     # expose a local https server
    ngrok tcp 22                     # tunnel arbitrary TCP traffic to port 22
    ngrok tls -hostname=foo.com 443  # TLS traffic for foo.com to port 443
    ngrok start foo bar baz          # start tunnels from the configuration file

VERSI

In [11]:
!npm install -g localtunnel


⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙⠹⠸⠼⠴⠦⠧⠇
added 22 packages in 2s
⠇
⠇3 packages are looking for funding
⠇  run `npm fund` for details
⠇

In [13]:
!streamlit run project/src/app.py &>/content/logs.txt &


In [14]:
!lt --port 8501


your url is: https://cold-worms-drum.loca.lt
/tools/node/lib/node_modules/localtunnel/bin/lt.js:81
    throw err;
    ^

Error: connection refused: localtunnel.me:5977 (check your firewall settings)
    at Socket.<anonymous> (/tools/node/lib/node_modules/localtunnel/lib/TunnelCluster.js:52:11)
    at Socket.emit (node:events:524:28)
    at emitErrorNT (node:internal/streams/destroy:169:8)
    at emitErrorCloseNT (node:internal/streams/destroy:128:3)
    at process.processTicksAndRejections (node:internal/process/task_queues:82:21)

Node.js v20.19.0


In [15]:
!curl ipv4.icanhazip.com


34.16.227.198


In [16]:
!wget -q https://github.com/cloudflare/cloudflared/releases/latest/download/cloudflared-linux-amd64
!chmod +x cloudflared-linux-amd64


In [17]:
!streamlit run project/src/app.py &>/content/logs.txt &


In [ ]:
!./cloudflared-linux-amd64 tunnel --url http://localhost:8501 --no-autoupdate


2025-11-26T13:38:03Z INF Thank you for trying Cloudflare Tunnel. Doing so, without a Cloudflare account, is a quick way to experiment and try it out. However, be aware that these account-less Tunnels have no uptime guarantee, are subject to the Cloudflare Online Services Terms of Use (https://www.cloudflare.com/website-terms/), and Cloudflare reserves the right to investigate your use of Tunnels for violations of such terms. If you intend to use Tunnels in production you should use a pre-created named tunnel by following: https://developers.cloudflare.com/cloudflare-one/connections/connect-apps
2025-11-26T13:38:03Z INF Requesting new quick Tunnel on trycloudflare.com...
2025-11-26T13:38:07Z INF +--------------------------------------------------------------------------------------------+
2025-11-26T13:38:07Z INF |  Your quick Tunnel has been created! Visit it at (it may take some time to be reachable):  |
2025-11-26T13:38:07Z INF |  https://mask-relationship-whale-adding.trycloudflare.